### 필요 패키지 설치

In [1]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 8.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 19.9 MB/s eta 0:00:00


### wandb 로그인


wandb에 로그인하여 api키를 입력합니다.

In [2]:
import wandb

# WanDB 로그인
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


True

### hyperparameter_tune

튜닝할 파라미터의 이름, 자료형, 범위 등 세팅합니다.

In [ ]:
# 파라미터 세팅 구현

sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'lg_max_iter' : {
            'values' : [300,500,700]
        },
        'lg_class_weight' : {
            'values' : [None,'balanced']
        },
        'rf_n_estimators': {
            'values': [50, 100, 200]
        },
        'rf_max_depth': {
            'values': [10, 20, 30]
        },
        'rf_min_samples_split': {
            'values': [2, 5, 10]
        },
        'dc_min_samples_split': {
            'values': [2, 4, 8]
        },
        'dc_max_depth': {
            'values': [10, 20, 40]
        },
        'knn_n_neighbors': {
            'values': [4, 6, 8]
        },
        'knn_leaf_size': {
            'values': [20, 30, 40]
        }
    }
}

sweep_id = wandb.sweep(
    sweep_config,
    project="hyperparameter_tune"
)
sweep_id

#### sweep 학습

In [ ]:
####임시로 학습 데이터 불러오는 블럭


In [ ]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

import numpy as np


def soft_voting(predictions):
    voting_result = np.argmax(np.sum(predictions, axis=0), axis=1)
    return voting_result


def wandb_training_function():
    with wandb.init() as run:
        params = wandb.config

        val_scores = list()

        lg = LogisticRegression(max_iter=params['lg_max_iter'],
                                class_weight=params['lg_class_weight'])

        # 모델 정의
        rf = RandomForestClassifier(n_estimators=params['rf_n_estimators'],
                                max_depth=params['rf_max_depth'],
                                min_samples_split=params['rf_min_samples_split'])

        dc = DecisionTreeClassifier(min_samples_split=params['dc_min_samples_split'],
                                    max_depth=params['dc_max_depth'])

        knn = KNeighborsClassifier(n_neighbors=params['knn_n_neighbors'],
                                    leaf_size=params['knn_leaf_size'])

        # 모델 리스트 구성
        models = [lg ,rf, dc, knn]

        # 모델 학습
        for model in models:
            model.fit(x_train, y_train)

        # 모델 예측
        preds_train = [model.predict_proba(x_train) for model in models]
        preds_valid = [model.predict_proba(x_valid) for model in models]
        voting_preds_train = soft_voting(preds_train)
        voting_preds_valid = soft_voting(preds_valid)

        # 모델 평가
        train_accuracy = accuracy_score(y_train, voting_preds_train)
        valid_accuracy = accuracy_score(y_valid, voting_preds_valid)

        val_scores.append(valid_accuracy)

        wandb.log({"train_accuracy": train_accuracy, "valid_accuracy": valid_accuracy})

        metrics = {"cv_metric": np.mean(val_scores)}
        wandb.log(metrics)

In [ ]:
count = 20

wandb.agent(sweep_id, function=wandb_training_function, count=count)

### 하이퍼 파라미터 분석

ensemble에 사용할 모델인 KNeighborsClassifier, DecisionTreeClassifier, RandomForestClassifier, LogisticRegression에 대한 hyperparameter 최적화를 진행하였음.
아래는 각 모델별 학습 그래프이다.

<center>
<img src='https://lh3.googleusercontent.com/fife/ALs6j_GneOSseWKIpQvuW8s7zZAm2Yhou-2Ai1vPsucvwI-8b4l09NovakHyatAICYkAQBFRHn8i5APBJVi2aSRMABk4It0nT0Ncri3STn8yfbJMEhzeNW-zBGWAf6rNtDRBkLFGcO4ID_bha-PaAaqLD7ILE_8xtlq_7mfbzBv4NlDbmpRUC4o6J13TQjvuziiZ1NMVnCh6_ZJ-V5e084D6UvRC_5IHHYRLpIYrCUIEn4t1UPixXV_bRME0eGB1XtTLWeY0Wyv-ymCtn1oR3nT0xiP-6vLODO-m-CfrKMutcMcZeum4OQb359IMtgQh1RmypOeNCC71py6d6sgxd7rBUQsnBL0Gd0JkaXeKHnqP2uTNOKVDoToq1VIs4L-ryE-IPDy11ACkCDg5U8P-j44LUz1ymmWbKAhFBG1gCWYTDpePC6HUduPg7q12wGADGjdjx-7_9w_UTs0uwPxE7QXhMlzG6xaR3Sj-vyA-cj7B7dHRDx_EwAZ2-7ev-6YwTpEKGTZb-nKRhDsxemPi0cgpHWk2aWQvn-azx1gi4EObOVa_Nc0yvhyTGKRQ9jp8qBNrSKo0ew5G7Q8LigRLGNRPYeco9lGdVP-7Bb3BXZ3htFsj_U5oyogwM9xqy0fefhx5N7jTsw-nxIY1DvqBzyXnqry2ne1GAUIWmsLS9S8-i3d8gbNh4NqLuNMoI54r41OIlSWMdLR726QJ8afjeg1KzyfhkveGg9UQhM0_dXwYUNwvUkPkeTUCngfDdSirs2gyP5cxfuKsD5YNYZ7MxXD6F5Y7bWSWvXMWV6AH88A4CtY6U_WsM1E7Ymt4qGkRgbML1qonNO9cWyA3izfwselEetieL5BBYM1rirgqfD5ghDAqnaOeyIX8w6Or17PfaHfJk_lhUq20S20tiAgWLcaeM9jvRZXodY1OMWCUheTaH9vpG5RaxFcSS1ShArSoXFU4qwl9iOiIRceZpllTXIHfuuVDKWZ_NxzRJHajZwL3FXOog_yXTML7NnDjnkVWoFlbIixHqebXaHS77c4TcFgZ3x2a75C4sBuYKvJi0RRjL9Jqp4AvEEBQOYQVDFOMKNphtykqacJtfYqOr-FQJX6X4YMd9UPUjxab9HxHckzZGt4lVGKpqFboDKRIxHUs1bzPUslNEyH2ZsyJSkghyjHRSb4n67ZlnJq_Yicb4zwQgMhDajRyrykStTeDPqPOMWJ7lPxZn0BUUpzPhiYRMqQ0-WHeZjCcU749cVDbVCfHq9J0MToneX5QThgfeh7CIraRI86mbK9dNr8ygPVqxA0zXRBD9-YOyKgQw6YxcDr_7GqcQP52MPaaZz4FzQJ2iXKndK16seS4NjOZCyt76q-PpcQCL95WzVLCB6wTLuhsWtvyAMHAsnfI6TLyxbVDzgWoD9xLXVA8x_Y5uRJ5cAo0mEU-5iTMfQ0GvRIZeZA44ltIM1FbKsUP1PvLdq13SNSfBMk2MHBwcYN0lPuXJQ4U2LwrqLxvfoX5NMUPL-vRP4gzOnX3BjFsQAsxuO1PXSe-fgRhiK8m1rfno8CwJT3KrOKxoyEpcMtrM8xV3tAs1154UKKZ6JJRrULxunwubAhG_hBsDJiPe6wssYtXf1g=w1920-h922'/><br>
<img src='https://lh3.googleusercontent.com/fife/ALs6j_FzXop28SHhTsBX7UGBZsMm-EI1TY3JSw4Ti7XeIbj2lGZQ2OL51DsDg2wjPDLjGAp18fCI3lB2d_ZP_7WUBH-0zT1uraHd8Mm3D4hgUcTnRLijl0fv8Bu7SbCuObNQbIdNXAVhEGuNrudPv1nRV3pga0nqa4kpT4ngBZtX4Henmj6lcJuZYk_O-yUWEMWQQ0rJ_9U35_IP3lx-8YtIw9-uLcoNo75ZW7_HeNP206K6YK7emUzQkjkqI3ibHrO6lPA-qtKYFTBr8TXvMgNR7FG0GZLUVlKlJkNvUKFPJXj3ZN3MevWDF2JqgNqNcQp9do6RVMx04y-HJavUN1LEvzUTR4p-sYZL5Cr5wnKWI0RndAIy5mMipIWJQdEI-w7mvr46Ps-QiX0oeoqE6XiGl-Hic-BFoTHU62eGH5tEP_aUX0CbDAJePAljqMJ3f8D4aRQLuldaTmrRAFhfNGbPtimYObNI0Z7v0YAVjmoR7JpOjtiGbYj1qIOT0NU5hDiMqkvTRoPZPEkvcBZ8nb5RFeNlynPttRzMwAugID3YhNhQoCnMn6Z5oAsJ-NJgEBxnTf0zk0UtzI4nYZO0e55-2wh7QUYI--r-0z3oU2v4hGwKZOhgDyTye_WfIJpuv4XYPr1hDYYroYZzR2zbCMXISPE1m0UDPeRLg_ReUrAqHEk5ejSMcRHH72tQ6zSnlofjS0L9fLWiZPBIWYTQb5vzT0GqmWJZ0JzPpweFvWpS_fils4PPA8OaMTGIeYdm-sehYo14vF7T1R5MwGyMMEMnNF78DYnZh79CrD2WXT3i16M_ITNIgTNtjK-k1oMxWpXRNJtsnoCFvA7FY0nvj2mATEVzu3E5jZEy6_Y-CxOHTr5dB0csV35kV01j9E3RkdO9zKkU7GIKel-QJYzjiQgf4IGFtJrzCtfA-fun8iMfxVTSGstvSo6g3wtfSB7zmXo4u2MQ6RXhGWFvSQKOzIqcWfxjkw5Lt5yVTXrVLdv5OlaNwpGlt6ERF5cvIs0xTD9muFqW_F8wokSfjdwpT8w9juzY4uv3cfL3WyetOosNOjDQqBdKU7IXS7VRegc9sZUGOYhPDNK6nsCD09WJuFHOnSWgfimlcLE3x3duj5cHzFh9iH3fzHeoobHnMnnX_VXgTf6zj5TbvjtlEpGbpexNqa2hQofeRgUdnNiCKI-A-86WY3H2MvyxeUwlsfb3lyKuZ8EvkyOZieuC5eXjMMkRBY6muhQMBT8tXeAVWsUb4v7Rl-ulaKmtcmjKua3MzPq-XCsrYb_g_tiFTf7ri6WfcY1EhKo25VXWXPl5BkJcsaTeVdU-_kPJxzXsthkAWrQ3teZ4_aeCQyoQ3WLoLTVyfXl4SJu5xbh3KzHHLXnMz6BM7gM01vCiH0orzEZrkEqax090ZeWXRMig1aRmWeRSsrK3uo9fuh9vLItNeUtvc8F71A-ZA5hsOte5TLI9eudh1I98Kd-m7tEJRoclUDbTSRQ8kzaauBiLSF5QxyvBrxXVQHcxApWb1Tg0QWW2QJys50LuQS0t67CXVlBWvBylcJoUMNFm9_hiG3SidzXzg-qz3qYmo06imhsvdUUane0ivzjl1WacW62AZvGb0xQ=w1600-h922'/><br>
<img src='https://lh3.googleusercontent.com/fife/ALs6j_Hw6x5g-rcVEtrg_vWpCda6EpnNs5S3OrF4UAJeRNmWwJDyOT2F4MG6ekK0uSPXnOVfwmBhxO3OJMPUC6JbHowP9sSBzB6rCm7uT6nvmY-Ufi-E62M3znWXVdaj89glbnPj-J6I7M71cOsdi2nqPFg0HfayVk_ZpS9QkbZuV-OlYi0AK0-HzBTv-Hxa_frNkAQXsSfi0_kUex-jQVVKftknaoKwfSDun4Q3kSeLhk-ixeaqcO8JNo4qTUGWX6A3uKGqKpowzrhyI6vBzrw5gVoKjU-kZaa-hXQ-ljSJz4VXKw1O8p4-YoP_AJYQue7LVOfpJn6XkaAJjxYNlhd7ZYfVI4XRsk8XVNsyb6HqVuk3CNN-joR8Q55m2qCWIC-w6Vh39UPS62fVR0-KS4yajzDV-xZqQOKKhFl99xDuOt5o_K01gExq0Z5K02FMV7nKlClInlSCZDhWLSbvIL1Gz6RgJlyNOp4v1bB4WhhWwqGmcR_JafHAfV2YtZ3vS-SqwltovVUYpU-T_KjTXrJtPdffkrWf4rAuzDs5vYSJOMXUUqG6PcLkVSXftfeC5OT2E6qnfAaH06UXhjS_1gOAzsIdodZEoaQgPZOWrkhPLrwR_bJ88kFTpBouJ23J2tgcwlNM0G-2qu7qP1r7nMTq2weBhpZqi5dhzjNop_csVME2iYR3HzPseUSa78I3f14vBgDQYu8WZ528iiVWPDGScg7KZK3CD3AdrDgrDHmFzaGA6TWlQE8mOH8aPhV8Xu2bm6_6smr_W-szA4omq2DSaUPYocEdo-pWkju50A3PbtFgYl_W8nUnNABQ45hJ69LTBf5EFnrP_NLfflfWWcyQTaEu1kDL81Vz4yTVjLsHsP3Pa4dxlvEN_YOSkZGHUZ3QF48amQgv9tMEo1mHEm6s9poutE8QKsGHT5-D179tcEDl4nE7WtHj3IFW9dJap4P4DT7sTazybhHFEaMF_vz_2WiL-WGAE-Ziq8pdXhkPKAngrXm_NqThlybkwlO8DwqTfK32dsbHyV2pmx9dAdsxI8rNk-3TBA7AQ7IU0NJLgwNNlzcLYpXC9XCf_axhnNPNah-Go9rLPN1SS7XAPq4ujnITocHz6WOtLBnuey0E7o2Ir2ul_fyesay6disyDw68a8HeY6jNlDYJYmnuLK9vBmG-qXWh9URn-EwAtw5g90Ux_PNX-xY3k9qRje5PJypTDVsxChPYMQbF5MD3-8kd7p8xXr1n6TPRq33j8Jg5eWd4Ix5Py-cvVbo8bJZ6kQ-XQxjnS0En6kALTZ_u_3rfs0XlZdAlf0E10kwhpRHYjF7wFFojd_hlIh3bumfBYPGHd3VZUP92ugH1zOzDmqLbx1p9AmbF2LmxxfkcuafzVtOWY6AOTrgDk7NfwVh3mSVOH-g58Z07mmocgMGn9CUaMWVCst2-LFrYEjDbujvhcaWYLpyDsoZU92ncBfJ53WWITtJL1X_edgRMDgr3ccsM-Acvi9EPpIc1A6Ue6JG9M1d7PMDXwf1SjPOSw64WDMCrdypoq4xI2E_uQykkUgthKIRtabZELwZnA2QnJAf9NBFOsYpAWUgQ8SWquwa2oIFE7rkbC9XfHf96Z52FH9KQQQ=w1920-h922'/><br>
<img src='https://lh3.googleusercontent.com/fife/ALs6j_HkJ0eeHNrnAv2ITJO9Lr0-BodPgV-ggU1x0hbowoVddx0W5QS4l30agx6kKw9n4cE6hy0-9JVguVbAC7x6aUFdYfv36RUzx0aRT6k5r2PLR5kh_lPKJNIgdoB8axAeUHLzVBYeAnXxsmEwCLStXWT0rhzjbvIoKBWuRufs8HksLjsTmrOUkTecrjt2rbggQGfybbcvcLjb9m3ydSafcluyz7y6qvXtUu0I-2gLR2fpRYggNa7S4mpavzpgqKtbTuoIczSPNiahI9dT-Zwl9QP94gAU6M-Kob7ZOCKtlQaWltch8AYzfrs2vHXXU4-68PyrEqFC6d4wdzh1O1Xjzx9X7F_SHNGofarIqbasNHJtwbrEidO7OTM-HTlxdqVi7OPitjZzjAxqjRSCcBBgMQdQsiN7V4OISIWRrmj1IiE5zzK1wyD007-xAe24v5tpFE0WK1h2JW85U_OuGKK4WXXvBybFXEm6DcUyktLQreaTNNB5JVSPr4LTcjrDpNpw6gRPbkD0ALROBL4qk4ML7qX5ZwShEXrGbEyJHlNhSrfSffRftBTTUMXRbmzJBnicP2qzSosoTkCo6lMkT0ERBW5mJFK2ywSZo_aIOgk7m5dpoDNcrFSMsnndAiH3hx2hauWGZ4jBLqCE8Jv5jDT88Soui9vmxvYkZubCpsH1cSztfjV9o-i-zEDPLIW46OWs5yW1nvF7LL_GQmfzoWv7Uq312-TJ9U15U_oBcDnnwlicEky87q1IzhGTdewEcOv4NSqnGgo_wDgHO7QhCsNpw2MUTL6pE1WzOwPYpGqDL1uOzD9zYFMSQa1NlBMyABYRu0xjJuqeODw-s1Gg_9jAG-eSpOAgMefEljLALJBvcoaom0MW2vJOfMoCMP-2xQjr4rDULs7G_pTmBroUjrMIgAciDwn_Ud4qltOChlRN4xLvLhJb1978SB2MFOcLgzjVD0vbd-L22qA8gvRE3vOZZBKTaOyMpEFgzhr5qC1Y3q6NgXPQTy7nJdbbzx4LWqKdLM-fgs-_scPfk9JVuxWSJdpmwIbiwtqU5z0F2tdT3tDc5jLO_zvZVu_qo9LrF6DXdLosUvR4Q8oxv0C2RAfa-N5vyDsD4k_v9amxoCxOWer-v55uqPWEqpxbjW00bSEdD55379_a92n_u0OZ74dQoyMhEnBDftERJfQbea6Q1oRnwo0GHCgKYFshexMirZsACLhs3AvyZDxTe-2ITnYieihn1MlnE38w9x-iBzpeLwxRIRS5IqAUi2fZP99JgxRmI0EHFB3uZgVN1ndQdzfr95LDYtl8Lk641wU9vGPmOX7H-bIx8DXcCPGtG-DghA3U8Tb8zbeVlVzoOewXT63q3e0p_Jq_-1VBsgbETWYBvOgEtDQCCWPisaT1gwcXN1rfeh9OF9VhcBpFfmYUf4AUDCsThovERbJL-p2DcMH96qFSHL-eooT7X_wZS_vc-LnirpZVpDHq5mgNr4M85vG_Y6oD2Z4PM4CKW2rY5_vZFFycyIbfd78VXFdW4eYm15yLGoVLGYMKIE0JvLDxsKjXcSasS5dAmzVLImXyBEhf1Gxm1CNmbgFl46KCtNOxT68GQp7yZGZRMWRXiUdqJWh0gw=w1600-h922'/><br>
<img src='https://lh3.googleusercontent.com/fife/ALs6j_H_0vOhrh9bIDx69G4YueFiNPLps-TPFgG-hdnr9zXVc2qHxuRHzmQ6r3Hwdlng2j-aKe9DcxGe4RsNF2wUsyEN7wH8KG9-vPGbR4Om2q5BRltKHm4kojt0rsuGmRHHzscAwmTaXUP0DAXnblxYgqbTS71ZKgEgrGQ08M-augH6ddVYeLG7dkDgaC7bvdS5xWXGsQc1-kuU-WbVJ6XjzkD0aJ_NyGbwOGLc_5xomX-uMyJM6OejVIFvfagFg-4eXSWxlv7xyq69vCX3ggUFGYpIMrbneyWFL0661ru1UIJh0BhOHun-UXWUL9qHjIKNd4Y7YPmjMyraCx2teNM9Ec_gb34a9jwG_NhhmCr1v_K_WBxMlhe_UCV1gYc0EEB211spXxIjiT33IrWchX0XKrTbXPBBsW8cNoquEaSnyf20QLLNQjmyChzLeC9sJJJuJj-bdCfp8W0nI5yZ7MTdrIBjSNwrgekyzBHzwM1oZv5gF1ylZUV6RtYr24qKOkqkVYGHo21W-g5dys84TQuck9EMBU1YV3MCrS7sT34bskGCc-yf_lv07r9kRJL9Blf-_LMzYhwNDXjnf3RAyNwEBInHnY6Ltf8SUM43H4_Bc341hDyKTkTlZDM4tuJ57KR7uQBilslBronzOBeRr3jG634TZAZI7iFMNuToAOwO2TivNRl-S5BvvcsT1TEMuuHxnXy8wjPSJzT3KC3Z9LFJomY11QFnRUkvorcG1yGFRlKrd6Ko7xn66_NXEkBZxaeWnmdNFuj_9M9dxd9bUlli67O4jpdf2wBsvjFC-5tvvoFHUdf1QywKohYhQ3OQjmgHSXbIJHXmq_OfoAfo1mbhDvVkJw68BaWBQCvkDOtimFjXm2FcIR6AyIOLEcZD2iVW2FXwlugWdlcFhXu_D81FiXOnP4EPUH3qV9NULx-lQYzR3l8hz7h_S-6nliEpR_bMv-IF58qAz0ut_wOZKJvCPzTX9TFXc4WoP-gqubjdDZ7KUUrEcq9rZJf1VCNtPbDBd1PYRuNhJOaMhOWTkSUqCbFS6whVIqhOGmyQBn4Dc4hGvF9WCmorGNcfFdliD7S1-ZJne33-fiBLbJBJrtU1cJX_ad9q9yQ4o2-apIEOOAxlEZBHOVj_dCxWm9Cquowh4jdZRMTXXjqVAs4Anuyjn3nTj8l_CDB_Vn40CneMmmKE_28gdW4tfWQ9LqmpVAx7LTOZDKphhJBGADSfCF2LuKeZXbko3aQlJQ1uTXax73Yap4lUK68Eqbr74-TQHJfaOin4WGFYRp8r_unSgMdnQoze9zALGf8pgdJV1SpTLwIlEIrC1CfjJxZqFCCm5scja7u5MxuhYwJEaKJfCHNynyTnIltb-WpHw85uHqXmi9GUEvoBAcxqfCzPACKKxq0lqqHMX_dOpkffqh_CAEa-FEWIqWFEyTf6urx-V2y8f6XN7NMzhwCjcYI1se-FFOPSYBmzFr2p0y4RL4hcRmVwOEIOY2ioSMjqJ_y6MnhiZxDj0Km5ZtKfxi14dUeIpeBVSIhFlT3NCgXFB4F4c21EWVjFysmfO-PrkevrXURrP4xKFqARyJlxdCEVgIvInIlMnI52J5LO_jKTiM7fA178HA=w1600-h922'/><br>

</center>

아래는 0.4224의 정확도를 가진 파라미터 값이다.

KNeighborsClassifier(leat_size = 20, n_neighbors = 4)<br>
DecisionTreeClassifier(max_depth = 20, min_samples_split=2)<br>
RandomForestClassifier(max_depth = 10,min_samples_split = 10, n_estimators = 50)<br>
LogisticRegression(class_weight=None, max_iter=300)

위 파라미터로 soft-vote-ensamble을 진행하였으나 이전보다 더 낮은 acc를 도출하였다.
